## 一、多项式朴素贝叶斯MultinomialNB

假设概率分布是服从一个**简单多项式**分布。多项式分布来源于统计学中的多项式实验，这种实验可以具体解释为：实验包括n次重复试验，每项试验都有不同的可能结果。在任何给定的试验中，特定结果发生的概率是不变的。

1.多项式分布擅长的是**分类型变量**，在其原理假设中，$P(x_i|Y)$的概率是离散的，并且不同$x_i$下的$P(x_i|Y)$相互独立，互不影响。虽然sklearn中的多项式分布也可以处理连续型变量，但现实中，如果我们真的想要处理**连续型变量**，我们应当使用**高斯朴素贝叶斯**。<br>
2.多项式实验中的实验结果都很具体，它所涉及的特征往往是**次数/频率/计数**出现与否这样的概念，这些概念都是离散的正整数，因此sklearn中的多项式朴素贝叶斯不接受**负值**的输入。

由于这样的特性，多项式朴素贝叶斯的特征矩阵经常是**稀疏矩阵**（不一定总是稀疏矩阵)，并且它经常被用于**文本分类**。可以使用著名的**TF-IDF**向量技术，也可以使用常见并且简单的**单词计数向量**手段与贝叶斯配合使用。

在$Y=c$这个分类下第$i$个特征所对应的参数$\theta_{ci}$：
$$\theta_{ci}=\frac{特征X_i在Y=c这个分类下的所有样本的取值总和}{所有特征在Y=c这个分类下的所有样本的取值总和}$$

`sklearn.naive_bayes.MultinomialNB`(alpha=1.0, fit_prior=True, class_prior=None)

**alpha**：浮点数，可不填<br>
拉普拉斯或利德斯通平滑的参数$\alpha$，如果设置为0则表示完全没有平滑选项。但是需要注意的是，平滑相当于人为给概率加上一些噪音，因此$\alpha$设置得越大，多项式朴素贝叶斯的精确性会越低（虽然影响不是非常大)，布里尔分数也会逐渐升高。

**fit_prior**：布尔值，可不填<br>
是否学习先验概率$P(Y=c)$。如果设置为false，则不适用先验概率，而使用统一先验概率，即认为每个标签类出现的概率是$\frac{1}{n\_classes}$。

**class_prior**：形似数组的结构，结构为(n_classes,)<br>
类的先验概率$P(Y=c)$。如果没有给出具体的先验概率则自动根据数据来进行计算。

**1.导入需要的模块和库**

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler #归一化
from sklearn.naive_bayes import MultinomialNB #多项式朴素贝叶斯
from sklearn.model_selection import train_test_split #训练集测试集分裂
from sklearn.datasets import make_blobs #自制数据
from sklearn.metrics import brier_score_loss #布里尔分数
from sklearn.preprocessing import KBinsDiscretizer #分箱（连续→分类）

**2.建立数据集**

In [2]:
class_1 = 500
class_2 = 500 #两个类别分别设定500个样本
centers = [[0.0, 0.0], [2.0, 2.0]] #设定两个类别的中心
clusters_std = [0.5, 0.5] #设定两个类别的方差
X, y = make_blobs(n_samples=[class_1, class_2],
                  centers=centers,
                  cluster_std=clusters_std,
                  random_state=0,
                  shuffle=False
                  )
X.shape

(1000, 2)

In [3]:
np.unique(y)

array([0, 1])

In [4]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=420)

**3.归一化，确保输入的矩阵不带有负数**

In [5]:
mms = MinMaxScaler().fit(Xtrain)
Xtrain_ = mms.transform(Xtrain)
Xtest_ = mms.transform(Xtest)

**4.建立多项式朴素贝叶斯分类器**

In [6]:
#拟合多项式朴素贝叶斯
mnb = MultinomialNB().fit(Xtrain_, Ytrain)

（1）属性class_log_prior_：根据数据获取的每个标签类的对数先验概率$log(P(Y))$。

In [7]:
mnb.class_log_prior_ #非常接近，没有样本不均衡问题

array([-0.69029411, -0.69600841])

In [8]:
mnb.class_log_prior_.shape

(2,)

In [9]:
np.unique(Ytrain)

array([0, 1])

In [10]:
(Ytrain == 1).sum()/Ytrain.shape[0] #标签为1的样本比例

0.49857142857142855

In [11]:
#可以使用np.exp来查看真正的概率值
np.exp(mnb.class_log_prior_)

array([0.50142857, 0.49857143])

（2）属性feature\_log\_prob\_：返回一个固定标签类别下的每个特征的对数概率$log(P(X_i|y))$。

In [12]:
mnb.feature_log_prob_ #2个特征，标签2个类别

array([[-0.76164788, -0.62903951],
       [-0.72500918, -0.6622691 ]])

In [13]:
mnb.feature_log_prob_.shape

(2, 2)

（3）属性class\_count\_：在fit时每个标签类别下包含的样本数。当fit接口中的sample_weight被设置时，该接口返回的值也会受到加权的影响。

In [14]:
mnb.class_count_

array([351., 349.])

In [15]:
mnb.class_count_.shape

(2,)

**5.分类器效果**

In [16]:
mnb.predict(Xtest_) #接口predict：返回标签

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
mnb.predict_proba(Xtest_) #接口predict_proba：每个样本在每个标签取值下的概率

array([[0.49847128, 0.50152872],
       [0.50065987, 0.49934013],
       [0.50122363, 0.49877637],
       [0.50183745, 0.49816255],
       [0.50146433, 0.49853567],
       [0.50153147, 0.49846853],
       [0.50204549, 0.49795451],
       [0.50033124, 0.49966876],
       [0.50105254, 0.49894746],
       [0.50182815, 0.49817185],
       [0.50270707, 0.49729293],
       [0.50133396, 0.49866604],
       [0.49820896, 0.50179104],
       [0.50342829, 0.49657171],
       [0.50099022, 0.49900978],
       [0.49974388, 0.50025612],
       [0.50423879, 0.49576121],
       [0.50449207, 0.49550793],
       [0.49818224, 0.50181776],
       [0.50245485, 0.49754515],
       [0.50393627, 0.49606373],
       [0.50193571, 0.49806429],
       [0.49996152, 0.50003848],
       [0.50460038, 0.49539962],
       [0.50261175, 0.49738825],
       [0.50140163, 0.49859837],
       [0.50332522, 0.49667478],
       [0.50122253, 0.49877747],
       [0.50409939, 0.49590061],
       [0.49998717, 0.50001283],
       [0.

In [18]:
mnb.score(Xtest_,Ytest) #接口score：准确度（不高）

0.5433333333333333

In [19]:
#布里尔分数：较高
brier_score_loss(Ytest,mnb.predict_proba(Xtest_)[:,1],pos_label=1)

0.24977828412546027

**6.效果不太理想，思考一下多项式贝叶斯的性质，应该做点什么？**

In [20]:
#把Xtiain转换成分类型数据
#注意Xtrain没有经过归一化，因为做哑变量之后自然所有的数据就不会有负数了
kbs = KBinsDiscretizer(n_bins=10, encode='onehot').fit(Xtrain) #独热，分10箱
Xtrain_ = kbs.transform(Xtrain)
Xtest_ = kbs.transform(Xtest)

In [21]:
Xtrain_.shape #2个特征每个特征分了10个箱得到哑变量

(700, 20)

In [22]:
mnb = MultinomialNB().fit(Xtrain_, Ytrain)
mnb.score(Xtest_,Ytest)

0.9966666666666667

In [23]:
brier_score_loss(Ytest,mnb.predict_proba(Xtest_)[:,1],pos_label=1)

0.0014593932778211862

可以看出，同样的数据，如果采用**哑变量**方式的**分箱**处理，多项式贝叶斯的效果会突飞猛进。

## 二、伯努利朴素贝叶斯BernoulliNB

多项式朴素贝叶斯可同时处理**二项分布**(抛硬币）和**多项分布**(掷骰子)，其中二项分布又叫做伯努利分布，它是一种现实中常见并且拥有很多优越数学性质的分布。因此，既然有着多项式朴素贝叶斯，我们自然也就又专门用来处理**二项分布**的朴素贝叶斯：伯努利朴素贝叶斯。<br>
伯努利贝叶斯类BernoulliN假设数据服从**多元伯努利分布**，并在此基础上应用朴素贝叶斯的训练和分类过程。多元伯努利分布简单来说，就是数据集中可以存在多个特征，但**每个特征都是二分类**的，可以以布尔变量表示，也可以表示为{0，1}或者{-1，1}等任意二分类组合。因此，这个类要求将样本转换为**二分类特征向量**，如果数据本身不是二分类的，那可以使用类中专门用来**二值化**的参数binarize来改变数据。

伯努利朴素贝叶斯与多项式朴素贝叶斯非常相似，都常用于处理**文本分类**数据。但由于伯努利朴素贝叶斯是处理二项分布，所以它更加在意的是“**存在与否**”，而不是“**出现多少次**”这样的次数或频率，这是伯努利贝叶斯与多项式贝叶斯的根本性不同。在文本分类的情况下，伯努利朴素贝叶斯可以使用**单词出现向量**（而不是单词计数向量）来训练分类器。**文档较短**的数据集上，伯努利朴素贝叶斯的效果会更加好。

`sklearn.naive_bayes.BernoulliNB`(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)

**alpha**：浮点数，可不填<br>
拉普拉斯或利德斯通平滑的参数$\alpha$，如果设置为0则表示完全没有平滑选项。但是需要注意的是，平滑相当于人为给概率加上一些噪音，因此$\alpha$设置得越大，多项式朴素贝叶斯的精确性会越低（虽然影响不是非常大)，布里尔分数也会逐渐升高。

**binarize**：浮点数或None，可不填<br>
将特征二值化的阈值，如果设定为None，则会假定特征已经被二值化完毕。

**fit_prior**：布尔值，可不填<br>
是否学习先验概率$P(Y=c)$。如果设置为false，则不适用先验概率，而使用统一先验概率，即认为每个标签类出现的概率是$\frac{1}{n\_classes}$。

**class_prior**：形似数组的结构，结构为(n_classes,)<br>
类的先验概率$P(Y=c)$。如果没有给出具体的先验概率则自动根据数据来进行计算。

In [24]:
from sklearn.naive_bayes import BernoulliNB

一般来说应该使用二值化的类`sklearn.preprocessing.Binarizer`来将特征一个个二值化，然而这样效率过低，因此选择**归一化**（不能输入负数）之后直接设置一个**阈值**。

In [25]:
mms = MinMaxScaler().fit(Xtrain)
Xtrain_ = mms.transform(Xtrain)
Xtest_ = mms.transform(Xtest)

（1）不设置二值化

In [26]:
bnl_ = BernoulliNB().fit(Xtrain_, Ytrain) #实例化+拟合
bnl_.score(Xtest_,Ytest)

0.49666666666666665

In [27]:
brier_score_loss(Ytest,bnl_.predict_proba(Xtest_)[:,1],pos_label=1)

0.25000009482193225

（2）设置二值化阈值为0.5

In [28]:
bnl = BernoulliNB(binarize=0.5).fit(Xtrain_, Ytrain) #二值化阈值0.5
bnl.score(Xtest_,Ytest)

0.9833333333333333

In [29]:
brier_score_loss(Ytest,bnl.predict_proba(Xtest_)[:,1],pos_label=1)

0.010405875827339534

和多项式贝叶斯一样，伯努利贝叶斯的结果也受到数据结构非常大的影响。因此，根据数据的模样选择贝叶斯，是贝叶斯模型选择中十分重要的一点。

## 三、探索贝叶斯：贝叶斯的样本不均衡问题

探讨一个分类算法永远都逃不过的核心问题：**样本不平衡**。贝叶斯由于分类效力不算太好，因此对样本不平衡极为敏感，接下来就来看一看样本不平衡如何影响了贝叶斯。

**1.导入需要的模块，建立样本不平衡的数据集**

In [30]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB #多项式、高斯、伯努利
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs #自制数据集
from sklearn.preprocessing import KBinsDiscretizer #分箱
from sklearn.metrics import brier_score_loss as BS,recall_score,roc_auc_score as AUC #布里尔分数、recall（自创二分类数据）、AUC

In [31]:
class_1 = 50000 #多数类为50000个样本
class_2 = 500 #少数类为500个样本
centers = [[0.0, 0.0], [5.0, 5.0]] #设定两个类别的中心
clusters_std = [3, 1] #设定两个类别的方差
X, y = make_blobs(n_samples=[class_1, class_2],
                  centers=centers,
                  cluster_std=clusters_std,
                  random_state=0,
                  shuffle=False
                 )

In [32]:
X.shape

(50500, 2)

In [33]:
np.unique(y)

array([0, 1])

**2.查看所有贝叶斯在样本不平衡数据集上的表现**

In [34]:
name = ["Multinomial","Gaussian","Bernoulli"]
models = [MultinomialNB(),GaussianNB(),BernoulliNB()]
for name,clf in zip(name,models):
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=420)
    
    #除了高斯朴素贝叶斯之外需要分箱。2个特征分10箱→20个特征
    if name != "Gaussian":
        kbs = KBinsDiscretizer(n_bins=10, encode='onehot').fit(Xtrain)
        Xtrain = kbs.transform(Xtrain)
        Xtest = kbs.transform(Xtest)
    
    clf.fit(Xtrain,Ytrain) #拟合
    y_pred = clf.predict(Xtest) #预测的分类结果
    proba = clf.predict_proba(Xtest)[:,1] #每个样本在每个特征上的概率
    score = clf.score(Xtest,Ytest) #准确率
    print(name)
    print("\tBrier:{:.3f}".format(BS(Ytest,proba,pos_label=1))) #布里尔分数
    print("\tAccuracy:{:.3f}".format(score)) #准确度
    print("\tRecall:{:.3f}".format(recall_score(Ytest,y_pred))) #召回率
    print("\tAUC:{:.3f}".format(AUC(Ytest,proba))) #AUC

Multinomial
	Brier:0.007
	Accuracy:0.990
	Recall:0.000
	AUC:0.991
Gaussian
	Brier:0.006
	Accuracy:0.990
	Recall:0.438
	AUC:0.993
Bernoulli
	Brier:0.009
	Accuracy:0.987
	Recall:0.771
	AUC:0.987


从结果上来看，**多项式朴素贝叶斯**判断出了所有的多数类样本，但放弃了全部的少数类样本，受到样本不均衡问题影响最严重。**高斯**比多项式在少数类的判断上更加成功一些，至少得到了43.8%的recall。**伯努利贝叶斯**虽然整体的准确度和布里尔分数不如多项式和高斯朴素贝叶斯和，但至少成功捕捉出了77.1%的少数类。可见，**伯努利贝叶斯最能够忍受样本不均衡问题**。<br>
可是，伯努利贝叶斯只能用于处理**二项分布**数据，在现实中，强行将所有的数据都二值化不会永远得到好结果，在有多个特征的时候，更需要一个个去判断究竟二值化的**阈值**该取多少才能够让算法的效果优秀。这样做无疑是非常低效的。<br>
那如果目标是捕捉少数类，应该怎么办呢？高斯朴素贝叶斯的效果虽然比多项式好，但是也没有好到可以用来帮助我们捕捉少数类的程度——43.8%，还不如抛硬币的结果。因此，孜孜不倦的统计学家们改进了朴素贝叶斯算法，修正了包括**无法处理样本不平衡**在内的传统朴素贝叶斯的众多缺点，得到了新兴贝叶斯算法：**补集朴素贝叶斯**。

## 四、改进多项式朴素贝叶斯：补集朴素贝叶斯ComplementNB

补集朴素贝叶斯（complement naive Bayes，CNB）算法是标准多项式朴素贝叶斯算法的改进。CNB的发明小组创造出CNB的初衷是为了解决贝叶斯中的“**朴素**”假设带来的各种问题，他们希望能够创造出数学方法以逃避朴素贝叶斯中的朴素假设，让算法能够不去关心所有特征之间是否是条件独立的。以此为基础，他们创造出了能够**解决样本不平衡**问题，并且能够一定程度上**忽略朴素假设**的补集朴素贝叶斯。在实验中，CNB的参数估计已经被证明比普通多项式朴素贝叶斯更稳定，并且它特别适合于样本不平衡的数据集。有时候，CNB在**文本分类**任务上的表现有时能够优于多项式朴素贝叶斯，因此现在补集朴素贝叶斯也开始逐渐流行。

CNB使用来自每个标签类别的**补集**的概率，并以此来计算每个特征的权重。$$\theta_{i,j≠c}=\frac{\alpha_i+\sum_{y_i≠c}x_{ij}}{\alpha_in+\sum_{i,y≠c}\sum_{i=1}^{n}x_{ij}}$$其中$j$表示每个样本，$x_{ij}$表示在样本$j$上对于特征$i$下的取值，在文本分类中通常是计数的值或TF-IDF值，$\alpha$是像标准多项式朴素贝叶斯中一样的平滑系数。其实就是多项式分布的逆向思路。

`sklearn.naive_bayes.ComplementNB`(alpha=1.0, fit_prior=True, class_prior=None, norm=False)

**alpha**：浮点数，可不填<br>
拉普拉斯或利德斯通平滑的参数α，如果设置为0则表示完全没有平滑选项。但是需要注意的是，平滑相当于人为给概率加上一些噪音，因此α设置得越大，多项式朴素贝叶斯的精确性会越低（虽然影响不是非常大)，布里尔分数也会逐渐升高。

**norm**：布尔值，可不填<br>
在计算权重的时候是否使用L2范式来规范权重的大小。默认不进行规范，即不跟从补集朴素贝叶斯算法的全部内容，如果希望进行规范，则设置True。TF-IDF通常默认为flase。

**fit_prior**：布尔值，可不填<br>
是否学习先验概率$P(Y=c)$。如果设置为false，则不适用先验概率，而使用统一先验概率，即认为每个标签类出现的概率是$\frac{1}{n\_classes}$。

**class_prior**：形似数组的结构，结构为(n_classes,)<br>
类的先验概率$P(Y=c)$。如果没有给出具体的先验概率则自动根据数据来进行计算。

In [35]:
from sklearn.naive_bayes import ComplementNB
from time import time
import datetime

In [36]:
name = ["Multinomial","Gaussian","Bernoulli","Complement"]
models = [MultinomialNB(),GaussianNB(),BernoulliNB(),ComplementNB()]
for name,clf in zip(name,models):
    times = time()
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=420)
    #预处理
    if name != "Gaussian":
        kbs = KBinsDiscretizer(n_bins=10, encode='onehot').fit(Xtrain)
        Xtrain = kbs.transform(Xtrain)
        Xtest = kbs.transform(Xtest)
    
    clf.fit(Xtrain,Ytrain)
    y_pred = clf.predict(Xtest)
    proba = clf.predict_proba(Xtest)[:,1]
    score = clf.score(Xtest,Ytest)
    print(name)
    print("\tBrier:{:.3f}".format(BS(Ytest,proba,pos_label=1)))
    print("\tAccuracy:{:.3f}".format(score))
    print("\tRecall:{:.3f}".format(recall_score(Ytest,y_pred)))
    print("\tAUC:{:.3f}".format(AUC(Ytest,proba)))
    print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

Multinomial
	Brier:0.007
	Accuracy:0.990
	Recall:0.000
	AUC:0.991
00:00:040924
Gaussian
	Brier:0.006
	Accuracy:0.990
	Recall:0.438
	AUC:0.993
00:00:020947
Bernoulli
	Brier:0.009
	Accuracy:0.987
	Recall:0.771
	AUC:0.987
00:00:034872
Complement
	Brier:0.038
	Accuracy:0.953
	Recall:0.987
	AUC:0.991
00:00:036900


可以发现，补集朴素贝叶斯牺牲了部分整体的**精确度**和**布里尔分数**，但是得到了十分高的**召回率**Recall，捕捉出了98.7%的少数类，并且在此基础上维持了和原本的多项式朴素贝叶斯一致的AUC分数。和其他的贝叶斯算法比起来，我们的补集朴素贝叶斯的运行速度也十分优秀。如果目标是**捕捉少数类**，则选择**补集朴素贝叶斯**。